In [1]:
#make sure plots are embedded into the notebook
%pylab inline 

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pywikibot as pwb
import pandas as pd
import networkx as nx

import pylab as pl

# Make plots prettyful
pl.style.use('fivethirtyeight')

VERBOSE:pywiki:Starting 1 threads...


Misspelled?
Please see https://www.mediawiki.org/wiki/Special:MyLanguage/Manual:Pywikibot/Windows
Defaulting to family='test' and mylang='test'.


In [3]:
langs = [
    ('en', 'English'), 
    ('de', 'German'),
    ('es', 'Spanish'),
    ('fr', 'French'),
    ('ja', 'Japanese'),
    ('ru', 'Russian'),
    ('it', 'Italian'),
    ('pt', 'Portuguese'),
    ('zh', 'Chinese'),
    ('nl', 'Dutch'),
    ('pl', 'Polish'),
    ('ar', 'Arabic'),
    ('tr', 'Turkish'),
    ('sv', 'Swedish'),
    ('ko', 'Korean'),
    ('fa', 'Persian'),
    ('uk', 'Ukrainian'),
    ('he', 'Hebrew'),
    ('id', 'Indonesian'),
    ('cs', 'Czech'),
    ('ur', 'Urdu'),
    ('hi', 'Hindi')
]

lang_codes = [i[0] for i in langs]

In [4]:
cities = pd.read_csv('cities.csv')

In [5]:
cities.columns

Index([u'ar', u'cs', u'de', u'en', u'es', u'fa', u'fr', u'he', u'hi', u'id',
       u'it', u'ja', u'ko', u'nl', u'pl', u'pt', u'ru', u'sv', u'tr', u'uk',
       u'ur', u'zh'],
      dtype='object')

In [6]:
subset100 = cities[cities['en'].isin([
    u'Shanghai',
    u'Karachi',
    u'Lagos',
    u'Istanbul',
    u'Tokyo',
    u'Mumbai',
    u'Moscow',
    u'São Paulo',
    u'Seoul',
    u'New York City'
])][['en','zh','tr','ja','ru','pt','ko','ur','hi','de']]

In [7]:
subset25 = cities[cities['en'].isin([
    u'Shanghai',
    u'Istanbul',
    u'Tokyo',
    u'Moscow',
    u'New York City'
])][['en','zh','tr','ja','ru']]

In [8]:
subset25

,en,zh,tr,ja,ru
0,Shanghai,上海市,Şanghay,上海市,Шанхай
4,Istanbul,伊斯坦堡,İstanbul,イスタンブル,Стамбул
5,Tokyo,東京都,Tokyo,東京都,Токио
9,Moscow,莫斯科,Moskova,モスクワ,Москва
20,New York City,纽约,New York,ニューヨーク,Нью-Йорк


In [9]:
# Queue to process pages
# Keep tuples of (title,depth)
# Where depth is the current depth, don't go past depth
queue = []
queued = {}

def scrapeCity(graph,lang,city,cities,max_depth=3):
    if max_depth < 1:
        return
    
    site = pwb.Site(lang, 'wikipedia')
    
    queue.append((pwb.Page(site, city),0))
    queued[city] = None
    
    while queue:
        page, depth = queue.pop()
        title = page.title()
        
        #if depth < max_depth:
        for p in page.linkedPages(namespaces=0):
            if p.namespace():
                continue
                
            t = p.title()

            graph.add_edge(title,t)

            # If we haven't queued/processed the page yet
            # And it isn't one of the cities.
            if t not in queued and not (t == cities).any():
                if depth < (max_depth - 1):
                    queued[t] = None
                    queue.append((pwb.Page(site,t), depth+1))
                

In [ ]:
for lang in subset25.columns:
    queue = []
    queued = {}
    df = nx.DiGraph()
    
    for city in subset25[lang]:
        print lang,city
        scrapeCity(df, lang, city, subset25[lang], max_depth=2)
        
    nx.write_gpickle(df,'%s25.pickle' % lang)

VERBOSE:pywiki:Found 1 wikipedia:en processes running, including this one.
VERBOSE:pywiki:Pausing due to database lag: Waiting for 10.64.32.23: 12 seconds lagged
Sleeping for 6.0 seconds, 2015-12-09 14:16:36
INFO:pywiki:Sleeping for 6.0 seconds, 2015-12-09 14:16:36
